# Dieting Network

# Setup

In [1]:
import requests

auth_token = "maio_cf75a54b_d24c_4ca3_b9c2_4b90d9d61d6f"

def make_payload(x):
    # This will convert your torch tensor / numpy array
    # into a list of floats
    return {"solution": x.flatten().tolist()}

url = "https://competitions.aiolympiad.my/api/selection2_2025_day2/selection2_2025_dieting_network"

def post_answer(data: dict):
    response = requests.post(url=url, json=data, headers={"X-API-Key": auth_token})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to submit, status code is {response.status_code}\n{response.text}"

## Datasets

In [2]:
import torch
import torch.nn as nn
from sklearn.metrics import r2_score

with open("X_train.pt", "rb") as f: X_train = torch.load(f)
with open("y_train.pt", "rb") as f: y_train = torch.load(f)
with open("X_val.pt", "rb") as f: X_val = torch.load(f)
with open("y_val.pt", "rb") as f: y_val = torch.load(f)
with open("X_test.pt", "rb") as f: X_test = torch.load(f)

## Network

In [3]:
class FixedLinear(nn.Module):
    """
    Similar to a nn.Linear layer, just not trainable
    """

    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = torch.ones(out_features, in_features)

    def forward(self, x):
        return torch.mm(x, self.weight.t())
    
class PolyAct(nn.Module):
    def __init__(self, coefficients):
        super().__init__()
        # Ensure coefficients are a tensor and have 5 elements for degree-4 polynomial
        self.coefficients = torch.tensor(coefficients, dtype=torch.float32)

    def forward(self, x):
        # Input x shape: (batch_size, 5), where all 5 elements per sample are identical (sum of inputs)
        # Take the first element of each sample as s
        s = x[:, 0:1]  # Shape: (batch_size, 1)
        # Compute powers s^0, s^1, s^2, s^3, s^4
        powers = torch.cat([s**i for i in range(5)], dim=1)  # Shape: (batch_size, 5)
        # Apply coefficients to each power
        out = powers * self.coefficients  # Shape: (batch_size, 5)
        return out

In [4]:
class DietNetwork(nn.Module):
    def __init__(self, coefficients):
        super().__init__()
        self.layer1 = FixedLinear(8, 5)

        self.act1 = PolyAct(coefficients) # <- Replace me!

        self.layer2 = FixedLinear(5, 5)

        self.act2 = nn.Identity() # <- Replace me!

        self.layer3 = FixedLinear(5, 5)

        self.act3 = nn.Identity() # <- Replace me!

        self.layer4 = FixedLinear(5, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.act1(x)
        x = self.layer2(x)
        x = self.act2(x)
        x = self.layer3(x)
        x = self.act3(x)        
        x = self.layer4(x)
        return x

In [5]:
s_train = X_train.sum(dim=1, keepdim=True).numpy()
y_train_scaled = y_train.numpy() / 25.0

In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

poly_features = PolynomialFeatures(degree=4, include_bias=True)
s_train_poly = poly_features.fit_transform(s_train)

reg = LinearRegression(fit_intercept=False)
reg.fit(s_train_poly, y_train_scaled)
coefficients = reg.coef_.flatten()

In [7]:
model = DietNetwork(coefficients)
model.eval()

DietNetwork(
  (layer1): FixedLinear()
  (act1): PolyAct()
  (layer2): FixedLinear()
  (act2): Identity()
  (layer3): FixedLinear()
  (act3): Identity()
  (layer4): FixedLinear()
)

In [8]:
with torch.no_grad():
    y_train_pred = model(X_train)
    y_val_pred = model(X_val)
    
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    
    print(
        f"train / val R2: {train_r2:.4f} / {val_r2:.4f}"
    )

train / val R2: 0.7375 / 0.7630


In [9]:
with torch.no_grad():
    y_test_pred = model(X_test)

In [10]:
# Should be (400, 1)
y_test_pred.shape

torch.Size([400, 1])

In [12]:
post_answer(make_payload(y_test_pred))

{'status': 'SUCCESS',
 'message': 'Answer for challenge selection2_2025_dieting_network submitted successfully on 2025-06-14 10:28:53.144417+00:00. Total submissions is 6 / 100.'}

# What I did is that since there are only 5 trainable parameters, we can use a 4-th order polynomial fit.